In [31]:
from bs4 import BeautifulSoup
import requests
import re

In [32]:
# assign variables destination
url = "https://www.seek.com.au/data-analyst-jobs/in-All-Sydney-NSW/full-time?daterange=1&salaryrange=80000-200000&salarytype=annual&sortmode=ListedDate"

# Grab the details/ content of that url
req = requests.get(url)

# /?keywords=data+analyst&amp;where=All+Sydney+NSW&amp;daterange=1
# https://www.seek.com.au/data-analyst-jobs/in-All-Sydney-NSW?daterange=1

In [33]:
soup = BeautifulSoup(req.text, 'html.parser')
# Parsers convert the input into single entities known as tokens and further convert the tokens into a graph
# or a tree structure for processing

# Job titles

In [34]:
job_titles = []
levels = soup.find_all('article', {'class' : '_37iADb_'})
for level in levels:
    divs = level.find_all('a', {'class' : '_2S5REPk'})
    for div in divs:
        job_titles.append(div.text)
#print(job_titles)

# Hirer

In [35]:
hirer = []
# levels = soup.find_all('article', {'class' : '_37iADb_'})
for level in levels:
    hirer_divs = level.find_all('span', {'class' : '_3mgsa7- _15GBVuT _2Ryjovs'})
    for hirer_div in hirer_divs:
        hirer.append(hirer_div.text)

desiredwords = ['at ']
hirer = [x for x in hirer if any(word in x for word in desiredwords)]
#print(hirer)

# Job ids

In [36]:
mylist = []
job_ids_final = soup.find_all(attrs={"data-job-id": re.compile("^[0-9]{8}$")})
for jif in job_ids_final:
    mylist.append(jif["data-job-id"])
#print(mylist)

# Build URLs

In [37]:
# Put Python loop output in a list
# https://stackoverflow.com/questions/39322625/how-to-put-python-loop-output-in-a-list/39323039
base_url = 'https://www.seek.com.au/job/'
dest_url = []
for myl in mylist:
    dest_url.append(base_url + myl)
#print(dest_url)

# Scrape job details

In [38]:
#dest_url
test1000 = []
for dtl in dest_url:
    requestcheck = requests.get(dtl)
    parser = BeautifulSoup(requestcheck.text, 'html.parser')
    #test999s = parser.find_all('div', {'class' : 'FYwKg PrHFr _1EtT-_4'}, {'data-automation': "job-detail-work-type"})
    test999s = parser.find_all('div', {'class' : 'FYwKg PrHFr _1EtT-_4'})
    for t in test999s:
        test1000.append(t.text)
#pprint(test1000)

In [39]:
# splitting a list above into sublists after 'Full time' text
test1001 = [i.strip().split(' ') for i in ' '.join(test1000).split('Full Time') if len(i) > 0 ]
#print(test1001)

# https://stackoverflow.com/questions/47604449/how-to-split-a-list-of-strings-into-sublists-of-strings-by-a-specific-string-ele

In [40]:
# function to convert / combine list into string
from itertools import chain

def collapse(lst):
    return ' ' .join(chain.from_iterable(lst))

# out = collapse([["I","am"], ["trying", "to", "convert"], ["listoflist", "intoastring."]])
# print(out)

In [41]:
# Push list through the function to convert to string
out = collapse(test1001)
#print(out)

In [42]:
# Converting string back to list after the text 'Sydney'
jobdetails = [i.strip() for i in out[6:].split('Sydney') if not i.isspace()]
#jobdetails

# Putting it all together

In [43]:
for one, two, three, four in zip(job_titles, hirer, jobdetails, dest_url):
    final = print(one, two, "|| Details: " + three + "|| Learn more at: " + four)

Data Engineer at JobAdder Pty Ltd || Details: CBD, Inner West & Eastern Suburbs Information & Communication Technology Database Development & Administration|| Learn more at: https://www.seek.com.au/job/52549006
Business Analyst - Lender Product at Lendi || Details: CBD, Inner West & Eastern Suburbs Information & Communication Technology Business/Systems Analysts|| Learn more at: https://www.seek.com.au/job/52364136
Marketing Automation Specialist at Macquarie Group Limited || Details: CBD, Inner West & Eastern Suburbs Marketing & Communications Digital & Search Marketing|| Learn more at: https://www.seek.com.au/job/52564059
Data and Analytics Manager at KBR - Kellogg Brown & Root Pty Ltd || Details: CBD, Inner West & Eastern Suburbs Information & Communication Technology Management|| Learn more at: https://www.seek.com.au/job/52564040
Digital Marketing Specialist at Hempel (Wattyl) Australia || Details: North West & Hills District Marketing & Communications Digital & Search Marketing||

In [44]:
# https://nbconvert.readthedocs.io/en/latest/removing_cells.html
# Removing cell in html

from traitlets.config import Config
import nbformat as nbf
from nbconvert.exporters import HTMLExporter
from nbconvert.preprocessors import TagRemovePreprocessor

# Setup config
c = Config()

# Configure tag removal - be sure to tag your cells to remove  using the
# words remove_cell to remove cells. You can also modify the code to use
# a different tag word
c.TagRemovePreprocessor.remove_cell_tags = ("remove_cell",)
c.TagRemovePreprocessor.remove_all_outputs_tags = ('remove_output',)
c.TagRemovePreprocessor.remove_input_tags = ('remove_input',)
c.TagRemovePreprocessor.enabled = True

# Configure and run out exporter
c.HTMLExporter.preprocessors = ["nbconvert.preprocessors.TagRemovePreprocessor"]

exporter = HTMLExporter(config=c)
exporter.register_preprocessor(TagRemovePreprocessor(config=c),True)

# Configure and run our exporter - returns a tuple - first element with html,
# second with notebook metadata
output = HTMLExporter(config=c).from_filename("a-SEEK_scrape_test.ipynb")

# Write to output html file
with open("outputreadytoemail.html",  "w", encoding='utf-8') as f:
    f.write(output[0])

In [45]:
# https://stackoverflow.com/questions/37592778/exporting-single-ipython-jupyter-notebook-cell-output